In [362]:
import numpy as np
import pandas as pd 
from datetime import datetime
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [363]:
full_data = pd.read_csv('downloads/cleaned_saffronart - clean_saffronart.csv', encoding = "ISO-8859-1")
full_data.dropna(thresh=1, inplace = True) #drop blank rows

In [364]:
columns_to_include = ['artist name', 'birth date', 'birth place', 'death date', 
                      'title', 'winning bid','low est', 'high est','auction date', 'category', 'style', 'details']

basic_info = full_data.loc[:, columns_to_include].copy()

#rename the column 'artist name' to 'artist':
basic_info.rename(columns = {'artist name': 'artist'}, inplace = True)

#reformat the column 'winning bid':
basic_info['winning bid'] = basic_info['winning bid'].str.lstrip("$").str.rstrip("\r").str.replace(",", "")
basic_info['low est'] = basic_info['low est'].str.lstrip("$").str.rstrip("\r").str.replace(",", "")
basic_info['high est'] = basic_info['high est'].str.lstrip("$").str.rstrip("\r").str.replace(",", "")

#convert string into numeric type:
basic_info['winning bid'] = pd.to_numeric(basic_info['winning bid'])
basic_info['low est'] = pd.to_numeric(basic_info['low est'])
basic_info['high est'] = pd.to_numeric(basic_info['high est'])

basic_info['size'] = (basic_info['details'].str.split('|').str[-2]).str.replace(r"\(.*\)","")
basic_info['medium'] = basic_info['details'].str.split('|').str[-3]

#reformat artist, category, and style:
for col in ["artist", "category", "style"]:
    basic_info[col] = basic_info[col].str.split().str.join(' ')
    basic_info[col] = basic_info[col].str.lower()

basic_info.head(5)

,artist,birth date,birth place,death date,title,winning bid,low est,high est,auction date,category,style,details,size,medium
0,akhilesh,1956,Indore,NaN,Divine Conversation,9000.0,5000.0,7000.0,12-13 JUNE 2019,painting,abstract,"Signed, inscribed and dated twice 'akhilesh/ 0...",71.5 x 71.5 in,Acrylic on canvas
2,akhilesh,1956,Indore,NaN,In search of Untitled forms -II,1120.0,5000.0,6670.0,12-13 NOVEMBER 2013,painting,abstract,Signed and dated in English (verso) | 2006 | A...,32.5 x 44 in,Acrylic on canvas
4,akhilesh,1956,Indore,NaN,In Search of Untitled Lines,6186.0,6900.0,8625.0,7-8 AUGUST 2013,painting,NaN,Signed and dated in English (verso) | 2006 | A...,47 x 47 in,Acrylic on canvas
6,akhilesh,1956,Indore,NaN,Magadhi,2400.0,5770.0,7695.0,27-28 FEBRUARY 2013,painting,abstract,Signed and dated in English (verso) | 2002 | A...,33 x 44.5 in,Acrylic on canvas
8,akhilesh,1956,Indore,NaN,Untitled,2942.0,6735.0,8655.0,27-28 FEBRUARY 2013,painting,abstract,Signed and dated in English (verso) | 2006 | A...,40 x 40 in,Acrylic on canvas


In [365]:
def trans_full_form(date_text, formats_list, index):
    #transform date string in full formats like 'june-2019' or '2019'
    
    if index == len(formats_list): #if all given formats were tried
        raise ValueError #not transformable 
    
    try:
        return datetime.strptime(date_text, formats_list[index])
    
    except ValueError: #if wrong format, try next format
        return trans_full_form(date_text, formats_list, index+1)
    
    except TypeError: #if nan, just pass
        return 
    
    
def trans_short_form(date_text, formats_list, index, lowerbound, upperbound):
    #transform date string in short formats like '7-apr-55'
    
    if index == len(formats_list):
        raise ValueError #not transformable
    
    try:
        date = datetime.strptime(date_text, formats_list[index])
    
    except ValueError: #if wrong format, try next format
        date = trans_short_form(date_text, formats_list, index+1, lowerbound, upperbound)
    
    except TypeError: #if nan, just pass
        return
         
    #because it defaults to the latest date, we need to check the lower and upper bounds
    #for example, '7-apr-55' is understood as April 7, 2055 instead of 1955
    
    if date > upperbound:
        return date.replace(year = date.year - 100) #subtract 100 years
    
    elif date < lowerbound:
        return date.replace(year = date.year + 100) #add 100 years

    return date

In [366]:
#convert the column auction date (str) into date:

basic_info['auction date'] = basic_info['auction date'].str.split().str[-2:]
basic_info['auction date'] = basic_info['auction date'].str.join('-').str.lower()

first_auction = datetime(2000, 1, 1)
latest_auction = datetime(2019, 7, 1)

for i, date in enumerate(basic_info['auction date']):
    try: #full format
        basic_info.iat[i, 8] = trans_full_form(date, ['%B-%Y'], 0)
    except ValueError: #try short format: 
        basic_info.iat[i, 8] = trans_short_form(date, ['%d-%b-%y'], 0, first_auction, latest_auction)

In [367]:
#convert the column birth date (str) into date:

first_birth = datetime(1769, 1, 1)
latest_birth = datetime(1988, 1, 1)

for i, date in enumerate(basic_info['birth date']):
    try: #full format
        basic_info.iat[i, 1] = trans_full_form(date, ['%Y', '%B %d, %Y'], 0)
    except ValueError: #try short format: 
        basic_info.iat[i, 1] = trans_short_form(date, ['%d-%b-%y'], 0, first_birth, latest_birth)

In [368]:
#convert the column death date (str) into date:

first_death = datetime(1837, 1, 1)
latest_death = datetime(2019, 3, 21)

for i, date in enumerate(basic_info['death date']):
    try: #full format
        basic_info.iat[i, 3] = trans_full_form(date, ['%Y', '%B %d, %Y'], 0)
    except ValueError: #try short format: 
        basic_info.iat[i, 3] = trans_short_form(date, ['%d-%b-%y'], 0, first_death, latest_death)

In [369]:
def get_age(death_date, birth_date):
    return (death_date - birth_date).days//364


def check_age(age, death_date, birth_date):    
    oldest = 101.0
    youngest = 24.0
    
    if age > oldest:
        death_date = death_date.replace(year = death_date.year - 100) #subtract 100 years
        age = age - 100
    
    elif age < youngest:
        birth_date = birth_date.replace(year = birth_date.year + 100) #add 100 years
        age = age + 100
        
    return age, death_date, birth_date

In [370]:
basic_info['age of artist'] = np.nan
now = datetime(2019, 7, 1) #the date the data was collected

for i in range(len(basic_info.index)):
    try:
        age = get_age(basic_info.iat[i, 3], basic_info.iat[i, 1]) #age = get_age(death date, birth date)
        
        age, death_date, birth_date = check_age(age, basic_info.iat[i, 3], basic_info.iat[i, 1]) #check_age()
        
        basic_info.iat[i, 14] = age
        basic_info.iat[i, 3] = death_date
        basic_info.iat[i, 1] = birth_date
        
    except TypeError: 
        try: #if death date is NaN
            basic_info.iat[i, 14] = get_age(now, basic_info.iat[i, 1]) #age = get_age(death date, birth date)
            
        except TypeError: #if birth date is also NaN
            pass

In [371]:
basic_info['medium'] = basic_info['medium'].str.lower()
basic_info['size'] = basic_info['size'].str.lower()
basic_info = basic_info[basic_info['size'].str.contains('x') == True]

In [372]:
basic_info = basic_info.reset_index(drop=True)

In [373]:
basic_info['size'] = basic_info["size"].apply(lambda x: ''.join([" " if ord(i) < 32 or ord(i) > 126 else i for i in x]))
basic_info['size'] = basic_info['size'].str.rstrip()

In [374]:
df = basic_info[['artist', 'birth date', 'birth place', 'death date', 'age of artist', 'title',
       'winning bid', 'low est', 'high est', 'auction date', 'category',
       'style', 'size', 'medium']]

In [375]:
df.to_csv("saffronart_dataset.csv", index = False)